In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats
from tqdm import tqdm

import tinkoffpy as tf

from datetime import datetime, timedelta, date
import datetime
import retentioneering

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# pio.templates.default = "plotly_dark"

/Users/r.yunisov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/r.yunisov/Library/Python/3.9/lib/python/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/r.yunisov/Library/Python/3.9/lib/python/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it

------
# Step - 2.
------

In [2]:
df_1 = tf.gp_to_df("""
WITH cte AS (select *,
                    case
                        when date_diff is null AND valid_to_dt != '5999-01-01' then 'left'
                        when date_diff >= 30 AND date_diff <= 90 then 're_active'
                        WHEN date_diff < 30 OR valid_to_dt = '5999-01-01' then 'current'
                        else 'others'
                        end                                                                start_status,
                    ROW_NUMBER() OVER (PARTITION BY gender_cd ORDER BY gender_cd)       AS row_number_by_gender,
                    ROW_NUMBER() OVER (PARTITION BY lvn_state_nm ORDER BY lvn_state_nm) AS row_number_by_region
             FROM (SELECT *,
                          ABS((valid_to_dt - future_end::DATE)) AS date_diff
                   FROM (select *,
                                lead(valid_from_dt)
                                over (partition by party_rk, financial_account_subtype_cd order by valid_to_dt) as future_end
                         from (SELECT party_rk,
                                      financial_account_subtype_cd,
                                      active_flg,
                                      valid_from_dt,
                                      valid_to_dt,
                                      first_date_by_product_client,
                                      (CURRENT_DATE - first_active_date)       AS days_after_first_activation,
                                      (CURRENT_DATE - first_active_date) / 365 AS years_after_first_activation,
                                      age,
                                      gender_cd,
                                      children_cnt,
                                      non_resident_flg,
                                      last_mb_session_dttm,
                                      last_ib_session_dttm,
                                      last_session_dttm,
                                      last_used_product_cd,
                                      monthly_income_amt,
                                      lvn_state_nm
                               FROM (SELECT a.party_rk,
                                            financial_account_subtype_cd,
                                            active_flg,
                                            valid_from_dt,
                                            valid_to_dt,
                                            MIN(valid_from_dt)
                                            OVER (PARTITION BY a.party_rk, financial_account_subtype_cd) AS first_date_by_product_client,
                                            MIN(valid_from_dt) OVER (PARTITION BY a.party_rk)            AS first_active_date,
                                            age,
                                            gender_cd,
                                            children_cnt,
                                            non_resident_flg,
                                            last_mb_session_dttm,
                                            last_ib_session_dttm,
                                            last_session_dttm,
                                            last_used_product_cd,
                                            monthly_income_amt,
                                            lvn_state_nm
                                     FROM prod_v_EMART.CLIENT_ACTIVE_STATUS a
                                              LEFT JOIN prod_v_emart.person_party b
                                                        ON a.party_rk = b.party_rk) full_table
                               WHERE active_flg != 0) as t) as tt) ttt),
     cte2 AS (SELECT party_rk,
                     financial_account_subtype_cd,
                     LEAD(financial_account_subtype_cd)
                     OVER (PARTITION BY party_rk ORDER BY valid_from_dt,valid_to_dt)                                          AS next_product,
                     financial_account_subtype_cd || ' - ' || LEAD(financial_account_subtype_cd)
                                                              OVER (PARTITION BY party_rk ORDER BY valid_from_dt,valid_to_dt) AS product_concat,
                     active_flg,
                     valid_from_dt,
                     valid_to_dt,
                     first_date_by_product_client,
                     next_dttm::DATE,
                     start_status,
                     age,
                     gender_cd,
                     children_cnt,
                     lvn_state_nm,
                     monthly_income_amt,
                     non_resident_flg,
                     date_diff,
                     days_after_first_activation,
                     years_after_first_activation
              FROM (SELECT party_rk,
                           financial_account_subtype_cd,
                           active_flg,
                           valid_from_dt,
                           valid_to_dt,
                           LEAD(valid_from_dt)
                           OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt) AS next_dttm,
                           first_date_by_product_client,
                           age,
                           children_cnt,
                           date_diff,
                           days_after_first_activation,
                           start_status,
                           gender_cd,
                           lvn_state_nm,
                           monthly_income_amt,
                           non_resident_flg,
                           years_after_first_activation
                    FROM cte) a
              WHERE valid_from_dt != next_dttm)
SELECT *,
       (CURRENT_DATE - first_date_by_product_client) AS days_after_first_activation_by_product,
       (CURRENT_DATE - first_date_by_product_client) / 365 AS years_after_first_activation_by_product,
       CASE 
           WHEN (CURRENT_DATE - first_date_by_product_client) / 365 BETWEEN 0 AND 1 THEN '0-1'
           WHEN (CURRENT_DATE - first_date_by_product_client) / 365 BETWEEN 2 AND 4 THEN '2-4'
           WHEN (CURRENT_DATE - first_date_by_product_client) / 365 BETWEEN 5 AND 7 THEN '5-7'
           WHEN (CURRENT_DATE - first_date_by_product_client) / 365 >= 8 THEN '8 and over'
        END years_after_first_activation_by_product_groups, 
       (valid_to_dt - valid_from_dt)                                                     AS diff_active_status_days,
       CASE
           WHEN (valid_to_dt - valid_from_dt) BETWEEN 0 AND 30 THEN '0-30'
           WHEN (valid_to_dt - valid_from_dt) BETWEEN 31 AND 60 THEN '31-60'
           WHEN (valid_to_dt - valid_from_dt) BETWEEN 61 AND 90 THEN '61-90'
           WHEN (valid_to_dt - valid_from_dt) BETWEEN 91 AND 120 THEN '91-120'
           WHEN (valid_to_dt - valid_from_dt) BETWEEN 121 AND 200 THEN '121-200'
           WHEN (valid_to_dt - valid_from_dt) >= 201 THEN '201 and over'
           END                                                                              diff_active_status_days_groups,
       rank()
       OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt) AS outflow_number,
       CASE
           WHEN age BETWEEN 18 and 25 THEN '18-25'
           WHEN age BETWEEN 26 and 30 THEN '26-30'
           WHEN age BETWEEN 31 and 35 THEN '31-35'
           WHEN age BETWEEN 36 and 40 THEN '36-40'
           WHEN age BETWEEN 41 and 45 THEN '41-45'
           WHEN age BETWEEN 46 and 50 THEN '46-50'
           WHEN age BETWEEN 51 and 55 THEN '51-55'
           WHEN age >= 56 THEN '56 and over'
           END                                                                              age_groups,
       CASE
           WHEN rank()
                OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt) BETWEEN 1 AND 2
               THEN '1-2'
           WHEN rank()
                OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt) BETWEEN 3 AND 4
               THEN '3-4'
           WHEN rank()
                OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt) >= 5 THEN '5 and over'
           END                                                                              outflow_number_groups,
       CASE
           WHEN monthly_income_amt BETWEEN 10000 AND 30000 THEN '10.000-30.000'
           WHEN monthly_income_amt BETWEEN 30001 AND 60000 THEN '30.001-60.000'
           WHEN monthly_income_amt BETWEEN 60001 AND 100000 THEN '60.001-100.000'
           WHEN monthly_income_amt BETWEEN 100001 AND 150000 THEN '100.001-150.000'
           WHEN monthly_income_amt BETWEEN 150001 AND 200000 THEN '150.001-200.000'
           WHEN monthly_income_amt BETWEEN 200001 AND 250000 THEN '200.001-250.000'
           WHEN monthly_income_amt BETWEEN 250001 AND 300000 THEN '250.001-300.000'
           WHEN monthly_income_amt >= 300001 THEN '300.001 and over'
           END                                                                              monthly_income_amt_groups,
       CASE
           WHEN years_after_first_activation BETWEEN 0 AND 3 THEN '0-3'
           WHEN years_after_first_activation BETWEEN 4 AND 7 THEN '4-7'
           WHEN years_after_first_activation >= 8 THEN '8 and over'
           END                                                                              years_after_first_activation_groups
FROM cte2
WHERE start_status = 're_active'
  AND valid_from_dt >= '2022-01-01'                                        
""")

2023-06-27 10:54:50 - INFO - Query executed successfully. Result saved to DataFrame.


------------
# Step - 3. Исследуем аномалии, закономерности в данных на этом этапе, вычисляем топ связок продуктов, смотрим частоту по временному ряду (всего / на человека);
-----------

In [3]:
df_1.head(
    ).sort_values('valid_from_dt')

,party_rk,financial_account_subtype_cd,next_product,product_concat,active_flg,valid_from_dt,valid_to_dt,first_date_by_product_client,next_dttm,start_status,...,days_after_first_activation_by_product,years_after_first_activation_by_product,years_after_first_activation_by_product_groups,diff_active_status_days,diff_active_status_days_groups,outflow_number,age_groups,outflow_number_groups,monthly_income_amt_groups,years_after_first_activation_groups
3,861672897,CUR,CUR,CUR - CUR,1,2022-06-05,2022-07-03,2022-05-29,2022-08-04,re_active,...,394,1,0-1,28,0-30,1,18-25,1-2,NaN,0-3
2,758025234,CUR,CUR,CUR - CUR,1,2022-06-18,2022-08-08,2021-10-28,2022-09-14,re_active,...,607,1,0-1,51,31-60,1,56 and over,1-2,10.000-30.000,0-3
0,518911654,CUR,CUR,CUR - CUR,1,2023-02-19,2023-03-21,2020-07-22,2023-05-08,re_active,...,1070,2,2-4,30,0-30,4,18-25,3-4,30.001-60.000,0-3
4,973434578,BIL,CUR,BIL - CUR,1,2023-02-26,2023-04-09,2023-02-26,2023-02-28,re_active,...,121,0,0-1,42,31-60,1,18-25,1-2,10.000-30.000,0-3
1,677882773,BNP,NaN,NaN,1,2023-04-10,2023-04-22,2023-04-10,2023-06-12,re_active,...,78,0,0-1,12,0-30,1,56 and over,1-2,150.001-200.000,8 and over


## Обрабатываем данные и рисуем линейный график частоты миганий в разрезе по периодам. Смотрим период с 2022-01-01

In [4]:

df_1['valid_from_dt'] = pd.to_datetime(df_1["valid_from_dt"])
df_1['valid_to_dt'] = pd.to_datetime(df_1["valid_to_dt"])
df_1['year_month_from'] = df_1['valid_from_dt'].dt.strftime('%Y-%m')
df_1['year_month_to'] = df_1['valid_to_dt'].dt.strftime('%Y-%m')

for_chart_all = df_1.query("first_date_by_product_client < '2021-12-31'")

for_chart_date_from = for_chart_all.groupby(['year_month_from'], as_index=False)['date_diff'].count()
for_chart_date_to = for_chart_all.groupby(['year_month_to'], as_index=False)['date_diff'].count()

fig_1_1 = px.line(for_chart_date_from, x="year_month_from", y="date_diff", markers=True)
fig_1_1.add_scatter(x=for_chart_date_to['year_month_to'], y=for_chart_date_to['date_diff'])
fig_1_1.show()

## Обрабатываем данные, чтобы собрать сводную таблицу из сочетаний продуктов между собой и сравнить частоту миганий

In [5]:
df_1['next_product'] = df_1.next_product.fillna(df_1.financial_account_subtype_cd)

for_chart = df_1.rename(columns={'financial_account_subtype_cd':'last_product_before_inactive', 'next_product':'first_product_after_inactive'}) \
        .groupby(['last_product_before_inactive', 'first_product_after_inactive'], as_index=False)['party_rk'].nunique() \
        .rename(columns={'party_rk':'number_of_couples'}) \
        .sort_values('number_of_couples', ascending=False) \
        .reset_index(drop=True) \
        .head(100) \

for_chart['pairs_of_products'] = for_chart['last_product_before_inactive'] + ' - ' + for_chart['first_product_after_inactive']
df_1['pairs_of_products'] = df_1['financial_account_subtype_cd'] + ' - ' + df_1['next_product']

df_1.rename(columns={'financial_account_subtype_cd':'last_product_before_inactive', 'next_product':'first_product_after_inactive'}) \
        .groupby(['last_product_before_inactive', 'first_product_after_inactive'], as_index=False)['party_rk'].count() \
        .rename(columns={'party_rk':'number_of_couples'}) \
        .sort_values('number_of_couples', ascending=False) \
        .reset_index(drop=True) \
        .head(100) \
        .pivot_table(index='last_product_before_inactive', columns='first_product_after_inactive', values='number_of_couples', aggfunc='sum') \
        .fillna(0) \
        .astype(int) \
        .replace(0, '-')




first_product_after_inactive,BIL,BNP,BNS,CAR,CCR,CLA,CLN,CUR,DEP,DLG,IIS,ISF,LEG,SAV,SCL,TRD,VKR
last_product_before_inactive,,,,,,,,,,,,,,,,,
BIL,506456,8609,-,-,70181,-,3733,228000,4774,-,1068,20264,-,37079,-,16221,6718
BNP,11059,275954,-,-,24596,-,1787,37577,-,-,-,6575,-,9251,-,2941,9497
BNS,-,-,4993,-,-,-,-,-,-,-,-,-,-,-,-,-,-
CAR,-,-,-,1053,-,-,-,-,-,-,-,-,-,-,-,-,-
CCR,46326,10802,-,-,749762,1485,9046,223892,4172,-,1071,16057,-,31460,1166,14922,15951
CLA,-,-,-,-,-,1284,-,-,-,-,-,-,-,-,-,-,-
CLN,1027,-,-,-,3310,-,8694,2102,-,-,-,-,-,1018,-,-,-
CUR,204202,41780,-,-,362053,1319,6211,3711089,6025,-,2751,53699,-,131681,-,70153,51059
DEP,1039,-,-,-,1054,-,-,-,10517,-,-,-,-,3544,-,1103,-


### - Рисуем теперь график гистограмм по количеству продуктовых связок (частота миганий) уникальных по клиентам

In [6]:
fig = px.bar(for_chart.nlargest(20 , 'number_of_couples'), x='pairs_of_products', y='number_of_couples', text_auto='.2s') # Можно ввести кол-во, которое нужно посмотреть
fig.show()



In [7]:
print('Share of CUR-CUR: ', round(3000000 / for_chart['number_of_couples'].sum() * 100,2), '%')

Share of CUR-CUR:  43.35 %


## Отбираем топ 2 связок продуктов и смотрим как меняются частота миганий в зависимости от различных признаков (lifetime, возраст, пол..)

###  - Проверяем изменения на Lifetime по топ 2 продуктам-связок и частоте миганий

In [8]:
df_top_pairs = df_1[(df_1['pairs_of_products'] == 'CUR - CUR') | (df_1['pairs_of_products'] == 'CCR - CCR')]
df_top_pair_4_step = df_1[(df_1['pairs_of_products'] == 'CUR - CUR')]
x = tuple(range(16))

for_funnel_chart = df_top_pairs.groupby(['pairs_of_products', 'years_after_first_activation'], as_index=False)['party_rk'].count() \
            .rename(columns={'party_rk':'count'}) \
            .query(f"years_after_first_activation in {x}", engine='python')

df_top_pairs['date_diff_groups'] = pd.cut(df_top_pairs['date_diff'], 3)
df_top_pairs['date_diff_groups'] = df_top_pairs['date_diff_groups'].astype(object)

df_top_pairs.groupby(['pairs_of_products', 'years_after_first_activation'], as_index=False)['party_rk'].count() \
            .rename(columns={'party_rk':'count'}) \
            .sort_values(['count'], ascending=False) \
            .pivot_table(index='pairs_of_products', columns='years_after_first_activation', values='count', aggfunc='sum') \
            .fillna(0) \
            .astype(int)



/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/2217713509.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/2217713509.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



years_after_first_activation,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,23
pairs_of_products,,,,,,,,,,,,,,,,,,
CCR - CCR,39489,129468,179702,99607,100114,62689,46519,26226,13368,15113,14050,8899,7041,1997,1348,1586,18,2528
CUR - CUR,460998,1414888,782403,328309,246756,143094,101881,63818,32276,35805,35811,26411,20660,5056,3710,5210,50,3953


### - Рисуем воронку по данным измений частоты миганий в зависимости от lifetime, чтобы визуализировать результат

In [9]:
fig_1 = px.funnel(for_funnel_chart, x='years_after_first_activation', y='count', color='pairs_of_products')
fig_1.show()

### - Проверяем изменения на группе данных по кол-ву дней миганий (Неактивного статуса)

In [10]:
df_top_pairs.groupby(['pairs_of_products', 'date_diff_groups'], as_index=False)['party_rk'].count() \
            .pivot_table(index='pairs_of_products', columns='date_diff_groups', values='party_rk', aggfunc='sum')


date_diff_groups,"(29.94, 50.0]","(50.0, 70.0]","(70.0, 90.0]"
pairs_of_products,,,
CCR - CCR,437191,206975,105596
CUR - CUR,1959059,1072764,679266


### - Проверяем изменения на топ 10 записях по городам и частоте миганий

In [11]:
for_funnel_3 = df_top_pairs.groupby(['pairs_of_products', 'lvn_state_nm'], as_index=False)['party_rk'].count() \
            .sort_values(['party_rk', 'lvn_state_nm', 'pairs_of_products'], ascending=False) \
            .reset_index(drop=True) \
            .query("lvn_state_nm in ('Г МОСКВА', 'Г САНКТ-ПЕТЕРБУРГ', 'КРАСНОДАРСКИЙ КРАЙ', 'МОСКОВСКАЯ ОБЛ', 'НИЖЕГОРОДСКАЯ ОБЛ', 'НОВОСИБИРСКАЯ ОБЛ', 'ПЕРМСКИЙ КРАЙ', 'РЕСП БАШКОРТОСТАН', 'РЕСП ТАТАРСТАН', 'РОСТОВСКАЯ ОБЛ')") \
#             .pivot_table(index='pairs_of_products', columns='lvn_state_nm', values='party_rk', aggfunc='sum') \
#             .fillna(0) \
#             .astype(int)

fig_2 = px.funnel(for_funnel_3, x='lvn_state_nm', y='party_rk', color='pairs_of_products')
fig_2.show()

--------
# Step - 4. Погружаемся в CUR-CUR связку по продуктам и исследуем клиентов по различным признакам (Факторный анализ) и группам (Кластерный анализ). 
## Смотрим клиентов, которые ведут активность с 2022-01-01, но первый продукт у них появился <2021-12-31, чтобы исключить тех, кто пришел в выбранный для анализа период впервые и активировал продукт
## 'outflow_number' - номер мигания, какой по счету раз клиент попадает в статус неактивного за выбранный период


--------

In [12]:
print('Columns: ', list(df_top_pair_4_step.columns))

df_top_pair_4_step = df_top_pair_4_step.query("first_date_by_product_client < '2021-12-31'")

df_top_pair_4_step.head()

Columns:  ['party_rk', 'financial_account_subtype_cd', 'next_product', 'product_concat', 'active_flg', 'valid_from_dt', 'valid_to_dt', 'first_date_by_product_client', 'next_dttm', 'start_status', 'age', 'gender_cd', 'children_cnt', 'lvn_state_nm', 'monthly_income_amt', 'non_resident_flg', 'date_diff', 'days_after_first_activation', 'years_after_first_activation', 'days_after_first_activation_by_product', 'years_after_first_activation_by_product', 'years_after_first_activation_by_product_groups', 'diff_active_status_days', 'diff_active_status_days_groups', 'outflow_number', 'age_groups', 'outflow_number_groups', 'monthly_income_amt_groups', 'years_after_first_activation_groups', 'year_month_from', 'year_month_to', 'pairs_of_products']


,party_rk,financial_account_subtype_cd,next_product,product_concat,active_flg,valid_from_dt,valid_to_dt,first_date_by_product_client,next_dttm,start_status,...,diff_active_status_days,diff_active_status_days_groups,outflow_number,age_groups,outflow_number_groups,monthly_income_amt_groups,years_after_first_activation_groups,year_month_from,year_month_to,pairs_of_products
0,518911654,CUR,CUR,CUR - CUR,1,2023-02-19,2023-03-21,2020-07-22,2023-05-08,re_active,...,30,0-30,4,18-25,3-4,30.001-60.000,0-3,2023-02,2023-03,CUR - CUR
2,758025234,CUR,CUR,CUR - CUR,1,2022-06-18,2022-08-08,2021-10-28,2022-09-14,re_active,...,51,31-60,1,56 and over,1-2,10.000-30.000,0-3,2022-06,2022-08,CUR - CUR
10,642412963,CUR,CUR,CUR - CUR,1,2022-07-17,2022-09-03,2021-02-14,2022-10-24,re_active,...,48,31-60,1,26-30,1-2,30.001-60.000,4-7,2022-07,2022-09,CUR - CUR
21,552710759,CUR,CUR,CUR - CUR,1,2022-02-05,2022-04-04,2020-10-30,2022-05-06,re_active,...,58,31-60,1,31-35,1-2,30.001-60.000,0-3,2022-02,2022-04,CUR - CUR
25,758623236,CUR,CUR,CUR - CUR,1,2022-03-21,2022-05-18,2021-10-29,2022-07-21,re_active,...,58,31-60,1,18-25,1-2,NaN,0-3,2022-03,2022-05,CUR - CUR


### Смотрим графически топ 10 городов по кол-ву миганий

In [13]:
for_bar_chart_step_4 = df_top_pair_4_step.query("first_date_by_product_client < '2022-12-31'") \
                    .groupby('lvn_state_nm', as_index=False)['outflow_number'].mean() \
                    .sort_values('outflow_number', ascending=False) \
                    .head(15)

fig_3 = px.bar(for_bar_chart_step_4, x='lvn_state_nm', y='outflow_number')
fig_3.show()          

## Смотрим графически среднее кол-во миганий на человека за период с 2022-01-01 
#### Учитываются только те, у кого продукт уже был открыт ранее, чтобы исключить тех, кто мог активировать впервые в период анализа

In [14]:
for_line_chart_step_4 = df_top_pair_4_step.query("first_date_by_product_client < '2022-12-31'") \
                    .groupby('year_month_to', as_index=False)['outflow_number'].mean()

fig_4 = px.line(for_line_chart_step_4, x="year_month_to", y="outflow_number", markers=True)
fig_4.show()

### Смотрим кол-во и долю клиентов в разрезе номера мигания

In [15]:
funnel_by_outflow = df_top_pair_4_step.groupby('outflow_number', as_index=False).agg({'date_diff':'mean', 'party_rk':'nunique'}) \
                    .sort_values('party_rk', ascending=False)

funnel_by_outflow['total_qty'] = funnel_by_outflow['party_rk'].max()

funnel_by_outflow['share_of_clients'] = funnel_by_outflow['party_rk'] / funnel_by_outflow['total_qty'] * 100

funnel_by_outflow[['outflow_number', 'party_rk', 'share_of_clients', 'date_diff']].rename(columns={'party_rk':'clients'}).round({'share_of_clients':2, 'date_diff':1})





,outflow_number,clients,share_of_clients,date_diff
0,1,1719372,100.00,52.6
1,2,432198,25.14,52.1
2,3,85688,4.98,51.3
3,4,12301,0.72,50.3
4,5,1138,0.07,47.5
5,6,83,0.00,43.6
6,7,17,0.00,47.0
7,8,6,0.00,44.3
8,9,1,0.00,30.0


### Работаем с номером мигания (оттока), кол-вом дней неактивности и клиентами / разными группами. Базовый пример ниже

In [16]:
df_top_pair_4_step.groupby(['party_rk', 'outflow_number'], as_index=False)['date_diff'].sum() \
                    .sort_values(['party_rk', 'outflow_number']) \
                    .iloc[:50] \
                    .pivot_table(index='outflow_number', columns='party_rk', values='date_diff', aggfunc='sum')

party_rk,145,819,849,876,1000,1177,1286,1300,1487,1616,...,6557,7071,7866,8009,8898,9040,9116,9162,9273,9756
outflow_number,,,,,,,,,,,,,,,,,,,,,
1,62,56,42,52,88,42,69,31,89,50,...,59,31,39,76,80,86,32,40,30,31
2,<NA>,<NA>,86,50,35,<NA>,<NA>,43,<NA>,<NA>,...,<NA>,<NA>,50,<NA>,<NA>,<NA>,<NA>,<NA>,90,50
3,<NA>,<NA>,55,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,44,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Сводная таблица по возрасту клиентов и номеру мигания в разрезе по кол-ву дней оттока

In [17]:
df_top_pair_4_step.groupby(['age_groups', 'outflow_number'], as_index=False)['date_diff'].mean() \
                    .sort_values(['age_groups', 'outflow_number']) \
                    .pivot_table(index='outflow_number', columns='age_groups', values='date_diff', aggfunc='sum') \
                    .round(1)

age_groups,18-25,26-30,31-35,36-40,41-45,46-50,51-55,56 and over
outflow_number,,,,,,,,
1,52.8,52.6,52.6,52.5,52.6,52.5,52.6,52.8
2,52.4,52.3,52.3,51.8,52.0,51.8,51.9,52.0
3,51.7,51.3,51.3,51.1,51.3,51.1,51.5,50.8
4,50.7,49.7,50.6,50.2,49.8,50.5,50.3,50.2
5,47.4,46.9,48.4,48.3,45.0,47.1,48.6,49.3
6,40.9,41.6,48.0,43.2,41.4,43.2,45.0,48.2
7,46.5,32.0,48.0,48.7,30.0,47.7,30.0,75.5
8,30.0,<NA>,60.0,43.5,<NA>,<NA>,<NA>,59.0
9,<NA>,<NA>,<NA>,30.0,<NA>,<NA>,<NA>,<NA>


### Сводная таблица по lifetime в банке  и номеру мигания в разрезе по кол-ву дней оттока

In [18]:
df_top_pair_4_step.groupby(['years_after_first_activation_groups', 'outflow_number'], as_index=False)['date_diff'].mean() \
                    .sort_values(['years_after_first_activation_groups', 'outflow_number']) \
                    .pivot_table(index='outflow_number', columns='years_after_first_activation_groups', values='date_diff', aggfunc='mean') \
                    .round(1)


years_after_first_activation_groups,0-3,4-7,8 and over
outflow_number,,,
1,52.7,52.5,52.5
2,52.2,52.0,51.8
3,51.4,51.1,51.4
4,50.4,50.0,49.6
5,47.6,47.9,45.0
6,43.6,43.4,43.9
7,45.0,52.4,44.0
8,30.0,58.7,30.0
9,<NA>,30.0,<NA>


### Сводная таблица по lifetime в продукте и номеру мигания в разрезе по кол-ву дней оттока

In [19]:
df_top_pair_4_step.groupby(['years_after_first_activation_by_product_groups', 'outflow_number'], as_index=False)['date_diff'].mean() \
                    .sort_values(['years_after_first_activation_by_product_groups', 'outflow_number']) \
                    .pivot_table(index='outflow_number', columns='years_after_first_activation_by_product_groups', values='date_diff', aggfunc='mean') \
                    .round(1)

years_after_first_activation_by_product_groups,0-1,2-4,5-7,8 and over
outflow_number,,,,
1,52.7,52.6,52.7,52.7
2,52.1,52.1,52.0,52.0
3,51.5,51.3,51.1,51.7
4,50.3,50.3,49.6,50.8
5,46.4,48.0,47.4,49.1
6,45.0,43.1,43.1,42.0
7,40.2,49.5,53.0,30.0
8,<NA>,44.8,57.0,30.0
9,<NA>,<NA>,30.0,<NA>


### Сводная таблица по кол-ву активных дней клиентов за выбранный период и номеру мигания в разрезе по кол-ву дней оттока

In [20]:
df_top_pair_4_step.groupby(['diff_active_status_days_groups', 'outflow_number'], as_index=False)['date_diff'].mean() \
                    .sort_values('diff_active_status_days_groups', ascending=False) \
                    .pivot_table(index='outflow_number', columns='diff_active_status_days_groups', values='date_diff', aggfunc='sum') \
                    .round(0)

diff_active_status_days_groups,0-30,121-200,201 and over,31-60,61-90,91-120
outflow_number,,,,,,
1,53.0,51.0,50.0,53.0,52.0,51.0
2,53.0,50.0,49.0,52.0,51.0,51.0
3,52.0,49.0,46.0,51.0,50.0,49.0
4,51.0,48.0,40.0,50.0,48.0,49.0
5,48.0,47.0,<NA>,47.0,46.0,45.0
6,43.0,<NA>,<NA>,47.0,<NA>,<NA>
7,47.0,<NA>,<NA>,<NA>,<NA>,<NA>
8,44.0,<NA>,<NA>,<NA>,<NA>,<NA>
9,30.0,<NA>,<NA>,<NA>,<NA>,<NA>


### Сводная таблица по группам номеров мигания клиентов и номеру мигания в разрезе по кол-ву дней оттока

In [21]:
df_top_pair_4_step.groupby(['outflow_number_groups', 'outflow_number'], as_index=False)['date_diff'].mean() \
                    .sort_values(['outflow_number_groups', 'outflow_number']) \
                    .pivot_table(index='outflow_number', columns='outflow_number_groups', values='date_diff', aggfunc='sum') \
                    .round(0)


outflow_number_groups,1-2,3-4,5 and over
outflow_number,,,
1,53.0,<NA>,<NA>
2,52.0,<NA>,<NA>
3,<NA>,51.0,<NA>
4,<NA>,50.0,<NA>
5,<NA>,<NA>,48.0
6,<NA>,<NA>,44.0
7,<NA>,<NA>,47.0
8,<NA>,<NA>,44.0
9,<NA>,<NA>,30.0


# Смотрим кол-во и долю клиентов в разрезе номера мигания, чтобы изучить значимость

In [22]:
funnel_by_outflow = df_top_pair_4_step.groupby('outflow_number', as_index=False).agg({'date_diff':'mean', 'diff_active_status_days':'mean', 'party_rk':'nunique'}) \
                    .sort_values('party_rk', ascending=False)

# funnel_by_outflow['total_qty'] = funnel_by_outflow['party_rk'].shift(1)
funnel_by_outflow['total_qty'] = funnel_by_outflow['party_rk'].sum()

funnel_by_outflow['share_of_clients'] = funnel_by_outflow['party_rk'] / funnel_by_outflow['total_qty'] * 100

for_share_fig = funnel_by_outflow[['outflow_number', 'party_rk', 'share_of_clients', 'date_diff', 'diff_active_status_days']].rename(columns={'party_rk':'clients'}).round({'share_of_clients':2, 'date_diff':1, 'diff_active_status_days':1})

## Так как мы считаем мигающих клиентов теми, у кого есть следующая запись в таблице, то неактивного

for_share_fig

# fig_share = px.bar(for_share_fig, x='outflow_number', y='share_of_clients', text_auto='.2s')

# fig_share.show()
fig_share = px.funnel(for_share_fig, x='outflow_number', y='share_of_clients')
fig_share.show()


### Когорта активных клиентов¶

In [23]:
df_top_pair_4_step.groupby(['year_month_from','year_month_to'], as_index=False)['party_rk'].agg('nunique') \
                    .sort_values('party_rk', ascending=False) \
                    .pivot_table(columns='year_month_to', index='year_month_from', values='party_rk', aggfunc='sum') \
                    .fillna(0) \
                    .astype(int) \
                    .replace(0, '-')

year_month_to,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05
year_month_from,,,,,,,,,,,,,,,,,
2022-01,7701,62845,33920,15798,7868,4733,3850,2468,1740,1354,928,772,719,430,406,290,105
2022-02,-,4741,77066,36097,15066,7879,6071,3786,2425,2021,1239,1068,954,565,478,375,140
2022-03,-,-,8773,84991,32560,14154,9514,5513,3465,2696,1739,1404,1216,686,638,444,155
2022-04,-,-,-,5999,87219,33242,17322,9034,5240,3933,2382,1814,1610,873,796,563,226
2022-05,-,-,-,-,8315,85558,38374,15537,8176,5476,3176,2389,1796,1024,984,675,226
2022-06,-,-,-,-,-,6179,96553,34908,15102,8729,4783,3237,2586,1400,1210,819,267
2022-07,-,-,-,-,-,-,9348,87752,31680,14264,6978,4244,3201,1715,1426,976,314
2022-08,-,-,-,-,-,-,-,7846,87276,34071,13223,7614,5070,2476,2084,1443,485
2022-09,-,-,-,-,-,-,-,-,5793,86752,30338,13607,8221,3683,2961,2008,678


### Кол-во клиентов на последний день (в рамках месяца, значит, последний день был в этом месяце)

In [24]:
number_of_clients = df_top_pair_4_step.groupby(['year_month_to', 'outflow_number'], as_index=False)['party_rk'].agg('nunique') \
                    .sort_values('year_month_to', ascending=False)

number_of_clients['total'] = number_of_clients['party_rk'].sum()


number_of_clients['share_of_clients'] = number_of_clients['party_rk'] / number_of_clients['total'] * 100

for_fig_5 = number_of_clients.sort_values('share_of_clients', ascending=False).round(2).rename(columns={'party_rk':'clients', 'total':'total_clients'}) \
                                .sort_values('share_of_clients', ascending=False)


fig_5 = px.bar(for_fig_5, x='year_month_to', y='clients', color='outflow_number', text_auto='.2s')

fig_5.show()


# STEP - 5. Остаемся в связке продуктов CUR-CUR, но берем весь период. Нужно добавить следующие признаки в выгрузку:
- lifetime в продукте / банке;
- Дней активности после неактивности и наоборот;
- Какой по счету раз клиент оттекает;
- Посмотреть другие продукты, возможно зацепиться за них;
<!-- - Посмотреть сумму транзакций у неактивных клиентов, что оценить долю тех, кто тратить <1000 or >100 рублей, например и попадает в статус неактивного. -->

In [25]:
df_step_5 = tf.gp_to_df(
"""
WITH cte AS (SELECT party_rk,
                    financial_account_subtype_cd,
                    active_flg,
                    valid_from_dt,
                    valid_to_dt,
                    MIN(valid_from_dt) OVER (PARTITION BY party_rk)                                   AS first_date_by_client,
                    (valid_to_dt - valid_from_dt)                                                     AS active_date_diff,
                    LEAD(valid_from_dt)
                    OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt) AS next_active_dt
             FROM prod_v_EMART.CLIENT_ACTIVE_STATUS
             WHERE active_flg = 1),
     cte2 AS (SELECT *,

                     (CURRENT_DATE - first_date_by_client) / 12                                                              AS months_after_the_first_activation,
                     (next_active_dt::DATE - valid_to_dt)                                                                    AS next_inactive_date_diff,
                     LEAD(active_date_diff)
                     OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_from_dt)                       AS next_active_date_diff,
                     CASE
                         WHEN next_active_dt IS NULL AND valid_to_dt = '5999-01-01' THEN 'active'
                         WHEN next_active_dt IS NOT NULL THEN 'flashing'
                         WHEN next_active_dt IS NULL AND valid_to_dt != '5999-01-01'THEN 'left' 
                             END                                                                                    churn_status,
                     LEAD(financial_account_subtype_cd)
                     OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt)                                        AS next_active_product,
                     LEAD(financial_account_subtype_cd, 2)
                     OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt)                                        AS after_next_active_product,
                     financial_account_subtype_cd || '-' || LEAD(financial_account_subtype_cd)
                                                            OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt) AS two_product_bundles,
                     financial_account_subtype_cd || '-' || LEAD(financial_account_subtype_cd)
                                                            OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt) ||
                     '-' || LEAD(financial_account_subtype_cd, 2)
                            OVER (PARTITION BY party_rk ORDER BY valid_from_dt, valid_to_dt)                                 AS three_product_bundles
              FROM cte
              WHERE valid_from_dt != next_active_dt::DATE),
     cte3 AS (SELECT party_rk,
                     financial_account_subtype_cd,
                     CASE
                         WHEN next_active_product IS NULL THEN financial_account_subtype_cd
                         WHEN next_active_product IS NOT NULL THEN next_active_product
                         END                                                                            next_active_product_without_null,
                     CASE
                         WHEN after_next_active_product IS NULL THEN next_active_product
                         WHEN after_next_active_product IS NOT NULL THEN after_next_active_product
                         END                                                                            after_next_active_product_without_null,
                     valid_from_dt,
                     valid_to_dt,
                     first_date_by_client,
                     months_after_the_first_activation,
                     active_date_diff,
                     next_inactive_date_diff,
                     next_active_date_diff                                                           AS next_active_date_diff_after_inactive_dd,
                     RANK()
                     OVER (PARTITION BY party_rk, financial_account_subtype_cd ORDER BY valid_to_dt) AS outflow_number
              FROM cte2
              WHERE churn_status = 'flashing')
SELECT *,
       CASE
           WHEN months_after_the_first_activation BETWEEN 1 AND 3 THEN '1-3 months'
           WHEN months_after_the_first_activation BETWEEN 4 AND 7 THEN '4-7 months'
           WHEN months_after_the_first_activation BETWEEN 8 AND 12 THEN '8-12 months'
           WHEN months_after_the_first_activation BETWEEN 13 AND 16 THEN '13-16 months'
           WHEN months_after_the_first_activation BETWEEN 17 AND 20 THEN '17-20 months'
           WHEN months_after_the_first_activation BETWEEN 21 AND 24 THEN '21-24 months'
           WHEN months_after_the_first_activation BETWEEN 25 AND 28 THEN '25-28 months'
           WHEN months_after_the_first_activation BETWEEN 29 AND 33 THEN '29-33 months'
           WHEN months_after_the_first_activation >= 34 THEN 'over 34 months'
           END months_first_activation_groups,
       CASE
           WHEN active_date_diff BETWEEN 10 AND 30 THEN '10-30 days'
           WHEN active_date_diff BETWEEN 31 AND 60 THEN '31-60 days'
           WHEN active_date_diff BETWEEN 61 AND 90 THEN '61-90 days'
           WHEN active_date_diff BETWEEN 91 AND 130 THEN '91-130 days'
           WHEN active_date_diff BETWEEN 131 AND 160 THEN '131-160 days'
           WHEN active_date_diff BETWEEN 161 AND 190 THEN '161-190 days'
           WHEN active_date_diff BETWEEN 191 AND 230 THEN '191-230 days'
           WHEN active_date_diff BETWEEN 231 AND 260 THEN '231-260 days'
           WHEN active_date_diff BETWEEN 261 AND 290 THEN '261-290 days'
           WHEN active_date_diff BETWEEN 291 AND 330 THEN '291-330 days'
           WHEN active_date_diff BETWEEN 331 AND 360 THEN '331-360 days'
           WHEN active_date_diff >= 361 THEN 'over 361 days'
           END active_days_group,
       CASE
           WHEN next_inactive_date_diff BETWEEN 10 AND 30 THEN '10-30 days'
           WHEN next_inactive_date_diff BETWEEN 31 AND 60 THEN '31-60 days'
           WHEN next_inactive_date_diff BETWEEN 61 AND 90 THEN '61-90 days'
           WHEN next_inactive_date_diff BETWEEN 91 AND 130 THEN '91-130 days'
           WHEN next_inactive_date_diff BETWEEN 131 AND 160 THEN '131-160 days'
           WHEN next_inactive_date_diff BETWEEN 161 AND 190 THEN '161-190 days'
           WHEN next_inactive_date_diff BETWEEN 191 AND 230 THEN '191-230 days'
           WHEN next_inactive_date_diff BETWEEN 231 AND 260 THEN '231-260 days'
           WHEN next_inactive_date_diff BETWEEN 261 AND 290 THEN '261-290 days'
           WHEN next_inactive_date_diff BETWEEN 291 AND 330 THEN '291-330 days'
           WHEN next_inactive_date_diff BETWEEN 331 AND 360 THEN '331-360 days'
           WHEN next_inactive_date_diff >= 361 THEN 'over 361 days'
           END next_inactive_days_group,
       CASE
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 10 AND 30 THEN '10-30 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 31 AND 60 THEN '31-60 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 61 AND 90 THEN '61-90 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 91 AND 130 THEN '91-130 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 131 AND 160 THEN '131-160 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 161 AND 190 THEN '161-190 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 191 AND 230 THEN '191-230 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 231 AND 260 THEN '231-260 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 261 AND 290 THEN '261-290 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 291 AND 330 THEN '291-330 days'
           WHEN next_active_date_diff_after_inactive_dd BETWEEN 331 AND 360 THEN '331-360 days'
           WHEN next_active_date_diff_after_inactive_dd >= 361 THEN 'over 361 days'
           END next_active_days_after_inactive_group,
       CASE
           WHEN outflow_number = 1 THEN '1'
           WHEN outflow_number = 2 THEN '2'
           WHEN outflow_number >= 3 THEN '3 and over'
           END outflow_number_group
FROM cte3
""")

2023-06-27 11:37:51 - INFO - Query executed successfully. Result saved to DataFrame.


In [26]:
df_step_5['two_product_bundles'] = df_step_5['financial_account_subtype_cd'] + ' - ' + df_step_5['next_active_product_without_null']

df_step_5['three_product_bundles'] = df_step_5['financial_account_subtype_cd'] + ' - ' + df_step_5['next_active_product_without_null'] + ' - ' + df_step_5['after_next_active_product_without_null']

df_step_5['three_product_bundles'] = df_step_5['three_product_bundles'].fillna(df_step_5['two_product_bundles'])

df_step_5.head(10)

,party_rk,financial_account_subtype_cd,next_active_product_without_null,after_next_active_product_without_null,valid_from_dt,valid_to_dt,first_date_by_client,months_after_the_first_activation,active_date_diff,next_inactive_date_diff,next_active_date_diff_after_inactive_dd,outflow_number,months_first_activation_groups,active_days_group,next_inactive_days_group,next_active_days_after_inactive_group,outflow_number_group,two_product_bundles,three_product_bundles
0,21301,CCR,CCR,NaN,2012-06-05,2013-04-05,2009-09-01,420,304,175,<NA>,4,over 34 months,291-330 days,161-190 days,NaN,3 and over,CCR - CCR,CCR - CCR
1,352615,CCR,CCR,CCR,2019-04-24,2019-12-18,2008-05-23,459,238,5,776,15,over 34 months,231-260 days,NaN,over 361 days,3 and over,CCR - CCR,CCR - CCR - CCR
2,786947,CCR,CCR,NaN,2013-04-12,2013-07-11,2011-07-18,363,90,17,<NA>,12,over 34 months,61-90 days,10-30 days,NaN,3 and over,CCR - CCR,CCR - CCR
3,1210768,CCR,CCR,CCR,2012-05-24,2012-07-20,2010-07-23,393,57,7,98,3,over 34 months,31-60 days,NaN,91-130 days,3 and over,CCR - CCR,CCR - CCR - CCR
4,2719736,CCR,CCR,CCR,2011-06-06,2013-05-09,2011-06-06,367,703,2,1181,1,over 34 months,over 361 days,NaN,over 361 days,1,CCR - CCR,CCR - CCR - CCR
5,2393574,CCR,CCR,CCR,2018-10-08,2019-03-23,2011-05-13,369,166,4,146,8,over 34 months,161-190 days,NaN,131-160 days,3 and over,CCR - CCR,CCR - CCR - CCR
6,344201,CCR,CCR,CCR,2009-11-16,2009-12-21,2008-06-06,458,35,21,31,4,over 34 months,31-60 days,10-30 days,31-60 days,3 and over,CCR - CCR,CCR - CCR - CCR
7,342684,CCR,CCR,CCR,2008-07-29,2008-09-02,2008-04-16,462,35,14,35,3,over 34 months,31-60 days,10-30 days,31-60 days,3 and over,CCR - CCR,CCR - CCR - CCR
8,706456,CUR,CCR,CCR,2018-08-09,2018-10-03,2009-01-19,439,55,5,<NA>,6,over 34 months,31-60 days,NaN,NaN,3 and over,CUR - CCR,CUR - CCR - CCR
9,204948,CCR,CCR,CCR,2008-08-11,2009-03-24,2008-06-10,457,225,3,35,2,over 34 months,191-230 days,NaN,31-60 days,2,CCR - CCR,CCR - CCR - CCR


In [27]:
df_5 = df_step_5.query("two_product_bundles == 'CUR - CUR'")

df_5['valid_from_dt'] = pd.to_datetime(df_5["valid_from_dt"])
df_5['valid_to_dt'] = pd.to_datetime(df_5["valid_to_dt"])
df_5['year_from'] = df_5['valid_from_dt'].dt.strftime('%Y')
df_5['year_to'] = df_5['valid_to_dt'].dt.strftime('%Y')

for_fig_6 = df_5.groupby('three_product_bundles', as_index=False)['party_rk'].agg('nunique') \
    .sort_values('party_rk', ascending=False) \
    .head(10)

fig_6 = px.bar(for_fig_6, x='three_product_bundles', y='party_rk', text_auto='.2s')

fig_6.show()

/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/561925608.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/561925608.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/561925608.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [28]:
25038729 - 10362609

14676120

In [29]:
df_cur = df_5.query("three_product_bundles in ('CUR - CUR', 'CUR - CUR - CUR')")

df_cur.head()

,party_rk,financial_account_subtype_cd,next_active_product_without_null,after_next_active_product_without_null,valid_from_dt,valid_to_dt,first_date_by_client,months_after_the_first_activation,active_date_diff,next_inactive_date_diff,...,outflow_number,months_first_activation_groups,active_days_group,next_inactive_days_group,next_active_days_after_inactive_group,outflow_number_group,two_product_bundles,three_product_bundles,year_from,year_to
17,4631403,CUR,CUR,NaN,2022-11-14,2023-01-19,2012-04-09,341,66,13,...,4,over 34 months,61-90 days,10-30 days,NaN,3 and over,CUR - CUR,CUR - CUR,2022,2023
32,5018121,CUR,CUR,CUR,2022-08-05,2022-09-04,2012-06-01,336,30,93,...,11,over 34 months,10-30 days,91-130 days,10-30 days,3 and over,CUR - CUR,CUR - CUR - CUR,2022,2022
37,70323440,CUR,CUR,CUR,2020-07-25,2020-08-24,2013-12-02,291,30,27,...,8,over 34 months,10-30 days,10-30 days,91-130 days,3 and over,CUR - CUR,CUR - CUR - CUR,2020,2020
43,108824109,CUR,CUR,NaN,2022-08-27,2022-10-13,2021-08-18,56,47,187,...,2,over 34 months,31-60 days,161-190 days,NaN,2,CUR - CUR,CUR - CUR,2022,2022
49,128731117,CUR,CUR,CUR,2022-05-20,2022-10-08,2016-03-22,221,141,4,...,6,over 34 months,131-160 days,NaN,31-60 days,3 and over,CUR - CUR,CUR - CUR - CUR,2022,2022


In [30]:
df_cur.outflow_number.mean()

2.845088503188069

In [31]:
df_6 = df_5.query("three_product_bundles in ('CUR - CUR', 'CUR - CUR - CUR')") \
    .groupby('outflow_number', as_index=False)['party_rk'].agg('nunique') \
    .rename(columns={'party_rk':'clients'})

df_6['all_clients'] = df_6['clients'].max()

df_6['share'] = df_6['clients'] / df_6['all_clients'] * 100

for_fig_share_2 = df_6[['outflow_number', 'clients', 'share']].round(2).head(10)

# fig_share_2 = px.bar(for_fig_share_2, x='outflow_number', y='share', text_auto='.2s')

# fig_share_2.show()

fig_share = px.funnel(for_fig_share_2, x='outflow_number', y='share')
fig_share.show()

In [32]:
df_5.columns

Index(['party_rk', 'financial_account_subtype_cd',
       'next_active_product_without_null',
       'after_next_active_product_without_null', 'valid_from_dt',
       'valid_to_dt', 'first_date_by_client',
       'months_after_the_first_activation', 'active_date_diff',
       'next_inactive_date_diff', 'next_active_date_diff_after_inactive_dd',
       'outflow_number', 'months_first_activation_groups', 'active_days_group',
       'next_inactive_days_group', 'next_active_days_after_inactive_group',
       'outflow_number_group', 'two_product_bundles', 'three_product_bundles',
       'year_from', 'year_to'],
      dtype='object')

In [33]:
for_fig_7 = df_5.groupby('year_from', as_index=False)[['active_date_diff', 'next_inactive_date_diff', 'next_active_date_diff_after_inactive_dd', ]].agg('mean') \
    .round(2)

fig_7 = px.bar(for_fig_7, x='year_from', y='next_inactive_date_diff', text_auto='.2s')

fig_7.show()

In [34]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Активные дни (avg)', x=for_fig_7['year_from'], y=for_fig_7['active_date_diff'], text=for_fig_7['active_date_diff'].round(0)),
    go.Bar(name='Неативные дни (avg)', x=for_fig_7['year_from'], y=for_fig_7['next_inactive_date_diff'], text=for_fig_7['next_inactive_date_diff'].round(0)),
    go.Bar(name='Активные после неактивных дни (avg)', x=for_fig_7['year_from'], y=for_fig_7['next_active_date_diff_after_inactive_dd'], text=for_fig_7['next_active_date_diff_after_inactive_dd'].round(0)),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [35]:
for_fig_7

,year_from,active_date_diff,next_inactive_date_diff,next_active_date_diff_after_inactive_dd
0,2009,356.28,567.56,225.92
1,2010,356.21,443.93,261.07
2,2011,194.1,352.05,220.05
3,2012,232.82,268.3,221.9
4,2013,243.14,265.56,200.94
5,2014,193.01,253.69,178.29
6,2015,213.67,263.48,180.83
7,2016,194.53,246.41,160.8
8,2017,192.29,196.95,149.16
9,2018,174.16,157.3,136.86


In [36]:
for_fig_8 = df_5.groupby(['year_from', 'outflow_number_group'], as_index=False)['party_rk'].agg('nunique')

fig_8 = px.bar(for_fig_8, x='year_from', y='party_rk', color='outflow_number_group', text_auto='.2s')

fig_8.show()

In [38]:
df_5.groupby('months_first_activation_groups', as_index=False)['next_inactive_days_group'].agg('mean')

/var/folders/3g/twm5z59x18s28jmjj3c3ghfm0000gq/T/ipykernel_2928/1957679296.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,months_first_activation_groups
0,1-3 months
1,13-16 months
2,17-20 months
3,21-24 months
4,25-28 months
5,29-33 months
6,4-7 months
7,8-12 months
8,over 34 months
